In [ ]:
How to add minor and other notes to music21 chords? Does not function atm

In [2]:
#!pip install music21
#!pip install pygame

In [5]:
from music21 import *
from music21.midi.realtime import StreamPlayer
import numpy as np
import music21
from matplotlib import pyplot as plt
from ipywidgets import interact, fixed
import numpy as np
import random

# Random Jazz Chords

In [15]:
def random_jazz_chord():
    root = random.choice(["C", "D", "E", "F", "G", "A", "B"])
    quality = random.choice(["maj7", "min7", "dom7", "m7b5", "dim7", "aug7"])
    inversion = random.choice([0, 1, 2, 3])
    return harmony.ChordSymbol(f"{root}{quality}")

In [16]:
random_jazz_chord()

<music21.harmony.ChordSymbol Emin7>

In [17]:
num_chords = 10

In [18]:
jazz_chords = [random_jazz_chord() for _ in range(num_chords)]

In [19]:
for i in jazz_chords:
    i.writeAsChord = True

In [20]:
jazz_chords

[<music21.harmony.ChordSymbol Edom7: E G# B D>,
 <music21.harmony.ChordSymbol Bdim7: B D F A->,
 <music21.harmony.ChordSymbol Fdom7: F A C E->,
 <music21.harmony.ChordSymbol Amaj7: A C# E G#>,
 <music21.harmony.ChordSymbol Bdim7: B D F A->,
 <music21.harmony.ChordSymbol Bmaj7: B D# F# A#>,
 <music21.harmony.ChordSymbol Faug7: F A C# E->,
 <music21.harmony.ChordSymbol Emin7: E G B D>,
 <music21.harmony.ChordSymbol Gmaj7: G B D F#>,
 <music21.harmony.ChordSymbol Emin7: E G B D>]

In [22]:
stream1 = stream.Stream()
stream1.append(jazz_chords)
stream1.show('text')
stream1.show("midi")

{0.0} <music21.harmony.ChordSymbol Edom7: E G# B D>
{1.0} <music21.harmony.ChordSymbol Bdim7: B D F A->
{2.0} <music21.harmony.ChordSymbol Fdom7: F A C E->
{3.0} <music21.harmony.ChordSymbol Amaj7: A C# E G#>
{4.0} <music21.harmony.ChordSymbol Bdim7: B D F A->
{5.0} <music21.harmony.ChordSymbol Bmaj7: B D# F# A#>
{6.0} <music21.harmony.ChordSymbol Faug7: F A C# E->
{7.0} <music21.harmony.ChordSymbol Emin7: E G B D>
{8.0} <music21.harmony.ChordSymbol Gmaj7: G B D F#>
{9.0} <music21.harmony.ChordSymbol Emin7: E G B D>


In [24]:
# yay ! this plays the chords
StreamPlayer(stream1).play()

# Circle of Fifth

In [1]:
# Define a function to plot the circle of fifths
def plot_circle_of_fifths():
    circle_keys = ["C", "G", "D", "A", "E", "B", "F#", "Db", "Ab", "Eb", "Bb", "F"]
    circle_minor_keys = [key.Key(k).relative.name for k in circle_keys]
    
    # Define the angular position of each key
    angles = np.linspace(0, 2 * np.pi, len(circle_keys) + 1)[:-1]
    
    # Create a polar plot
    fig, ax = plt.subplots(subplot_kw={"polar": True})
    
    # Major keys
    for i, (angle, key_name) in enumerate(zip(angles, circle_keys)):
        ax.text(angle, 1.3, key_name, fontsize=20, ha="center", va="center")
        
    # Minor keys
    for i, (angle, key_name) in enumerate(zip(angles, circle_minor_keys)):
        ax.text(angle, 0.85, key_name.lower(), fontsize=12, ha="center", va="center")
    
    # Set the radius and background color
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_rticks([])
    ax.set_rlim(0, 1.5)
    ax.set_facecolor("lavender")
    plt.title("Circle of Fifths, Major Keys on the outside", y=1.1)
    plt.show()

In [2]:
# Call the function to plot the circle of fifths
plot_circle_of_fifths()

NameError: name 'key' is not defined

In [ ]:
# Function to extract chords from a music21 stream
def extract_chords(music_stream):
    return [c for c in music_stream.recurse().getElementsByClass('Chord')]

# Function to plot chords on a polar graph
def plot_chords_polar(chords, index):
    if not chords:
        print("No chords found.")
        return

    if index < 0 or index >= len(chords):
        print("Invalid index.")
        return

    chord = chords[index]

    # Convert pitches to pitch classes
    pitch_classes = [p.pitchClass for p in chord.pitches]

    # Prepare the polar plot
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

    # Set the direction of the polar plot
    ax.set_theta_direction(-1)
    ax.set_theta_zero_location("N")

    # Set the range of the polar plot
    ax.set_rlim([0, 12])
    ax.set_rticks([])

    # Plot the chord
    ax.scatter([np.deg2rad(p * 30) for p in pitch_classes], [p for p in pitch_classes], s=100)

    # Add labels to the plot
    for i, p in enumerate(pitch_classes):
        ax.annotate(music21.pitch.Pitch(p).nameWithOctave,
                    xy=(np.deg2rad(p * 30), p),
                    xytext=(np.deg2rad(p * 30), p + 1),
                    fontsize=12,
                    horizontalalignment='center')

    plt.title(f"Chord {index + 1}:{chord} {chord.commonName}")
    plt.show()

# Load a music file and extract chords
#filename = 'path/to/your/music/file.mid'
#music_stream = music21.converter.parse(filename)
chords = extract_chords(stream1)

# Display the interactive widget
interact(plot_chords_polar, chords=fixed(chords), index=(0, len(chords) - 1))


# Chord resolution exploration

## Useful python functions

In [62]:
def show_score(score, title = None):
    if title:
        score.metadata = music21.metadata.Metadata()
        score.metadata.title = title
    score.show('text')
    #score.show('midi')
    StreamPlayer(score).play()
    #score.show()

In [47]:
def create_progression_chords(chord_progression):
    l=[]
    for chord in chord_progression:
        #chord.Chord works for midi numbers!
        l.append(music21.chord.Chord(chord, quarterLength=1))
    return l

In [48]:
def create_progression_score(progression, title):
    score = music21.stream.Score()
    chords = music21.stream.Stream(create_progression_chords(progression))
    score.append(chords)
    show_score(score, title)

## Voice leading

In [77]:
# Voice leading
def voice_leading():
    score = music21.stream.Score()
    progression = ['C7', 'F']
    chords = music21.stream.Stream()
    for p in progression:
        chords.append(music21.chord.Chord(p, quarterLength=4))
    score.append(chords)
    show_score(score)

voice_leading()

AccidentalException: m is not a supported accidental type

## Tritone resolution:

In [63]:
def tritone_resolution():
    score = music21.stream.Score()
    progression = ['G7', 'C']
    chords = music21.stream.Stream()
    for p in progression:
        chords.append(music21.chord.Chord(p, quarterLength=4))
    score.append(chords)
    show_score(score)

tritone_resolution()

{0.0} <music21.stream.Stream 0x12d46b610>
    {0.0} <music21.chord.Chord G7>
    {4.0} <music21.chord.Chord C>


## Enclosure or surrounding:

In [64]:
def enclosure():
    score = music21.stream.Score()
    melody = music21.stream.Part()
    melody.append(music21.note.Note('D4', quarterLength=1))
    melody.append(music21.note.Note('B3', quarterLength=1))
    melody.append(music21.note.Note('C4', quarterLength=2))
    score.append(melody)

    progression = ['C']
    chords = music21.stream.Part()
    for p in progression:
        chords.append(music21.chord.Chord(p, quarterLength=4))
    score.append(chords)
    show_score(score)

enclosure()

{0.0} <music21.stream.Part 0x12d46b100>
    {0.0} <music21.note.Note D>
    {1.0} <music21.note.Note B>
    {2.0} <music21.note.Note C>
{4.0} <music21.stream.Part 0x12d50a2f0>
    {0.0} <music21.chord.Chord C>


In [52]:
## Chromatic approach

In [53]:
def chromatic_approach():
    score = music21.stream.Score()
    melody = music21.stream.Part()
    melody.append(music21.note.Note('B3', quarterLength=1))
    melody.append(music21.note.Note('C4', quarterLength=3))
    score.append(melody)

    progression = ['C']
    chords = music21.stream.Part()
    for p in progression:
        chords.append(music21.chord.Chord(p, quarterLength=4))
    score.append(chords)
    show_score(score)

chromatic_approach()

{0.0} <music21.stream.Part 0x13590c940>
    {0.0} <music21.note.Note B>
    {1.0} <music21.note.Note C>
{4.0} <music21.stream.Part 0x13590f820>
    {0.0} <music21.chord.Chord C>


In [54]:
## Delayed resolution

In [55]:
def delayed_resolution():
    score = music21.stream.Score()
    melody = music21.stream.Part()
    melody.append(music21.note.Note('G4', quarterLength=2))
    melody.append(music21.note.Note('A4', quarterLength=1))
    melody.append(music21.note.Note('G4', quarterLength=1))
    score.append(melody)

    progression = ['C', 'G']
    chords = music21.stream.Part()
    for p in progression:
        chords.append(music21.chord.Chord(p, quarterLength=2))
    score.append(chords)
    show_score(score)

delayed_resolution()

{0.0} <music21.stream.Part 0x13599db10>
    {0.0} <music21.note.Note G>
    {2.0} <music21.note.Note A>
    {3.0} <music21.note.Note G>
{4.0} <music21.stream.Part 0x13599e590>
    {0.0} <music21.chord.Chord C>
    {2.0} <music21.chord.Chord G>


# Human Feelings and chord progressions

## Happiness

In [65]:
def happiness():
    score = music21.stream.Score()
    progression = ['C', 'G', 'A-','F']
    chords = music21.stream.Stream(create_progression_chords(progression))
    score.append(chords)
    show_score(score)

happiness()

{0.0} <music21.stream.Stream 0x12d46ace0>
    {0.0} <music21.chord.Chord C>
    {1.0} <music21.chord.Chord G>
    {2.0} <music21.chord.Chord A->
    {3.0} <music21.chord.Chord F>


## Sadness

In [67]:
def sadness():
    score = music21.stream.Score()
    progression = ['A-', 'F', 'C', 'G']
    chords = music21.stream.Stream(create_progression_chords(progression))
    score.append(chords)
    show_score(score) 

sadness()

{0.0} <music21.stream.Stream 0x12d46b6d0>
    {0.0} <music21.chord.Chord A->
    {1.0} <music21.chord.Chord F>
    {2.0} <music21.chord.Chord C>
    {3.0} <music21.chord.Chord G>


## Melancholy

In [69]:
def melancholy():
    score = music21.stream.Score()
    progression = ['C', 'G', 'A-', 'F7']
    chords = music21.stream.Stream(create_progression_chords(progression))
    score.append(chords)
    show_score(score)

melancholy()

{0.0} <music21.stream.Stream 0x1323a3a60>
    {0.0} <music21.chord.Chord C>
    {1.0} <music21.chord.Chord G>
    {2.0} <music21.chord.Chord A->
    {3.0} <music21.chord.Chord F7>


## Nostalgia

In [70]:
def nostalgia():
    score = music21.stream.Score()
    progression = ['C', 'E7', 'F', 'G']
    chords = music21.stream.Stream(create_progression_chords(progression))
    score.append(chords)
    show_score(score)

nostalgia()

{0.0} <music21.stream.Stream 0x13599ef50>
    {0.0} <music21.chord.Chord C>
    {1.0} <music21.chord.Chord E7>
    {2.0} <music21.chord.Chord F>
    {3.0} <music21.chord.Chord G>


## Tension or suspense

In [71]:
# Tension or suspense
def tension_suspense():
    score = music21.stream.Score()
    progression = ['C', 'F', 'G', 'b']
    chords = music21.stream.Stream(create_progression_chords(progression))
    score.append(chords)
    show_score(score)

tension_suspense()

AccidentalException: m is not a supported accidental type

## Mystery

In [72]:
def mystery():
    score = music21.stream.Score()
    progression = ['D-', 'G', 'A-']
    chords = music21.stream.Stream(create_progression_chords(progression))
    score.append(chords)
    show_score(score)

mystery()

{0.0} <music21.stream.Stream 0x1323a3e50>
    {0.0} <music21.chord.Chord D->
    {1.0} <music21.chord.Chord G>
    {2.0} <music21.chord.Chord A->


## Romantic

In [73]:
# Romantic
def romantic():
    score = music21.stream.Score()
    progression = ['C', 'D-', 'G7', 'C7']
    chords = music21.stream.Stream(create_progression_chords(progression))
    score.append(chords)
    show_score(score)

romantic()

{0.0} <music21.stream.Stream 0x1323a1060>
    {0.0} <music21.chord.Chord C>
    {1.0} <music21.chord.Chord D->
    {2.0} <music21.chord.Chord G7>
    {3.0} <music21.chord.Chord C7>


## Uplifting or triumphant

In [74]:
def uplifting_triumphant():
    score = music21.stream.Score()
    progression = ['C', 'E-', 'F', 'G']
    chords = music21.stream.Stream(create_progression_chords(progression))
    score.append(chords)
    show_score(score)

uplifting_triumphant()

{0.0} <music21.stream.Stream 0x12d468dc0>
    {0.0} <music21.chord.Chord C>
    {1.0} <music21.chord.Chord E->
    {2.0} <music21.chord.Chord F>
    {3.0} <music21.chord.Chord G>


# Other examples 

In [18]:
# Happiness
create_progression_score(['C', 'G', 'a', 'F'], "Happiness 1: I - V - vi - IV")
create_progression_score(['C', 'F', 'G', 'C'], "Happiness 2: I - IV - V - I")

NameError: name 'create_progression_score' is not defined

In [171]:
# Sadness
create_progression_score(['a', 'F', 'C', 'G'], "Sadness 1: vi - IV - I - V")
create_progression_score(['c', 'G', 'c'], "Sadness 2: i - V - i")

AccidentalException: m is not a supported accidental type

In [82]:
# Melancholy
create_progression_score(['C', 'G', 'A-', 'F7'], "Melancholy 1: I - V - vi - IVmaj7")
create_progression_score(['C', 'A-', 'F', 'G'], "Melancholy 2: I - vi - IV - V")

{0.0} <music21.metadata.Metadata object at 0x13b069f00>
{0.0} <music21.stream.Stream 0x13b069fc0>
    {0.0} <music21.chord.Chord C>
    {1.0} <music21.chord.Chord G>
    {2.0} <music21.chord.Chord A->
    {3.0} <music21.chord.Chord F7>
{0.0} <music21.metadata.Metadata object at 0x13b057c10>
{0.0} <music21.stream.Stream 0x13a767e20>
    {0.0} <music21.chord.Chord C>
    {1.0} <music21.chord.Chord A->
    {2.0} <music21.chord.Chord F>
    {3.0} <music21.chord.Chord G>


In [85]:
# Nostalgia
create_progression_score(['C', 'E7', 'F', 'G'], "Nostalgia 1: I - III7 - IV - V")
create_progression_score(['C', 'A7', 'D-', 'G'], "Nostalgia 2: I - VI7 - ii - V")

AttributeError: 'function' object has no attribute 'seventh'

In [ ]:
# Tension or suspense
create_progression_score(['C', 'F', 'G7', 'Bdim'], "Tension 1: I - IV - V7 - vii°")
create_progression_score(['C', 'G', 'Bb', 'F'], "Tension 2: I - V - bVII - IV")

In [ ]:
# Mystery
create_progression_score(['d', 'G', 'a'], "Mystery 1: Dorian mode - i - IV - v")
create_progression_score(['C', 'F', 'C', 'Db'], "Mystery 2: I - IV - I - bII")

In [114]:
# Romantic
create_progression_score(['C7', 'd7', 'G7', 'C7'], "Romantic 1: I - ii7 - V7 - Imaj7")
create_progression_score(['C7', 'a7', 'd7', 'G7'], "Romantic 2: I - vi7 - ii7 - V7")

{0.0} <music21.metadata.Metadata object at 0x1323eb100>
{0.0} <music21.stream.Stream 0x1323ebb80>
    {0.0} <music21.chord.Chord C7>
    {1.0} <music21.chord.Chord D7>
    {2.0} <music21.chord.Chord G7>
    {3.0} <music21.chord.Chord C7>
{0.0} <music21.metadata.Metadata object at 0x1323d6ec0>
{0.0} <music21.stream.Stream 0x1353d1060>
    {0.0} <music21.chord.Chord C7>
    {1.0} <music21.chord.Chord A7>
    {2.0} <music21.chord.Chord D7>
    {3.0} <music21.chord.Chord G7>


In [ ]:
# Uplifting or triumphant
create_progression_score(['C', 'A', 'Dm', 'G', 'C'], "Triumphant 1: I - VI - ii - V - I")
create_progression_score(['C', 'Em', 'F', 'G'], "Triumphant 2: I - iii - IV - V")

# Minor, seventh, dominant Seventh, diminished Chord

In [80]:
def create_multi_chord_progression(root):
    # Define chords types
    minor_chord = [0, 3, 7]
    seventh_chord = [0, 4, 7]
    dominant_seventh_chord = [0, 4, 7, 10]
    diminished_chord = [0, 3, 6]

    # Create chords
    chords = [
        chord.Chord([note.Note(root).pitch.transpose(interval) for interval in minor_chord]),
        chord.Chord([note.Note(root).pitch.transpose(interval) for interval in seventh_chord]),
        chord.Chord([note.Note(root).pitch.transpose(interval) for interval in dominant_seventh_chord]),
        chord.Chord([note.Note(root).pitch.transpose(interval) for interval in diminished_chord]),
    ]

    return chords

In [81]:
root = "C4"
chords = create_multi_chord_progression(root)

# Create a stream and add the chords
s = stream.Stream()
for c in chords:
    s.append(c)

# Show the score
StreamPlayer(s).play()

# Other test 

In [74]:
# Define chord progressions for main human emotions
emotions_chord_progressions = {
    'happy': [['I', 'IV', 'V'], ['I', 'vi', 'IV', 'V'], ['I', 'IV', 'ii', 'V']],
    'sad': [['i', 'VI', 'III'], ['i', 'iv', 'V'], ['i', 'iv', 'VII']],
    'angry': [['i', 'bII', 'V'], ['i', 'bII', 'bVII'], ['i', 'bIII', 'bVII']],
    'calm': [['I', 'ii', 'IV'], ['I', 'vi', 'IV'], ['I', 'ii', 'V']],
}

In [75]:
# Function to convert chord progressions to music21 chords and play them
def play_chord_progression(tonality: key.Key, progression: list, play=True):
    s = stream.Stream()

    for numeral in progression:
        rn = roman.RomanNumeral(numeral, tonality)
        s.append(rn)

    if play:
        sp = midi.realtime.StreamPlayer(s)
        sp.play()
    else:
        return s

In [76]:
# Function to play chord progressions for a given emotion
def play_emotion_chord_progressions(emotion: str, key_str: str = 'C'):
    if emotion not in emotions_chord_progressions:
        print(f"Invalid emotion: {emotion}. Please choose from {list(emotions_chord_progressions.keys())}.")
        return

    progressions = emotions_chord_progressions[emotion]
    tonality = key.Key(key_str) if emotion in ['happy', 'calm'] else key.Key(key_str, 'minor')

    for p in progressions:
        play_chord_progression(tonality, p)
        print(f"Chord progression for {emotion}: {', '.join(p)}")

In [77]:
# Example usage
emotion = random.choice(list(emotions_chord_progressions.keys()))
play_emotion_chord_progressions(emotion)

Chord progression for sad: i, VI, III
Chord progression for sad: i, iv, V
Chord progression for sad: i, iv, VII


# Write to midi

In [111]:
def create_midi_from_chords(chords, filename, tempo=120, repeat=4):
    s = stream.Stream()
    s.append(tempo)
    for _ in range(repeat):
        for ch in chords:
            s.append(ch)
    #s.write('midi', fp=filename)